# Vanilla CF

In [ ]:
# | default_exp methods.vanilla

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
# | export
from __future__ import annotations
from relax.import_essentials import *
from relax.methods.base import CFModule
from relax.base import BaseConfig
from relax.utils import auto_reshaping, grad_update, validate_configs

Using JAX backend.


In [ ]:
#| hide
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from relax.ml_model import MLModule
from relax.data_module import load_data
from relax.ml_model import load_ml_module

In [ ]:
# | exporti
@auto_reshaping('x')
def _vanilla_cf(
    x: jnp.DeviceArray,  # `x` shape: (k,), where `k` is the number of features
    y_target: Array, # `y_target` shape: (1,)
    pred_fn: Callable[[Array], Array],  # y = pred_fn(x)
    n_steps: int,
    lr: float,  # learning rate for each `cf` optimization step
    lambda_: float,  #  loss = validity_loss + lambda_params * cost
    validity_fn: Callable,
    cost_fn: Callable,
    apply_constraints_fn: Callable
) -> jnp.DeviceArray:  # return `cf` shape: (k,)
    @jit
    def loss_fn_1(y_true: Array, y_pred: Array):
        return validity_fn(y_true, y_pred).mean()

    @jit
    def loss_fn_2(x: Array, cf: Array):
        return cost_fn(cf, x).mean()

    @partial(jit, static_argnums=(2,))
    def loss_fn(
        cf: Array,  # `cf` shape: (k, 1)
        x: Array,  # `x` shape: (k, 1)
        pred_fn: Callable[[Array], Array],
    ):
        y_pred = pred_fn(x)
        cf_y_true = 1.0 - y_pred
        cf_y_pred = pred_fn(cf)
        return loss_fn_1(cf_y_true, cf_y_pred) + lambda_ * loss_fn_2(x, cf)

    @loop_tqdm(n_steps)
    def gen_cf_step(
        i, cf_opt_state: Tuple[Array, optax.OptState] #x: Array, cf: Array, opt_state: optax.OptState
    ) -> Tuple[jnp.DeviceArray, optax.OptState]:
        cf, opt_state = cf_opt_state
        cf_grads = jax.grad(loss_fn)(cf, x, pred_fn)
        cf, opt_state = grad_update(cf_grads, cf, opt_state, opt)
        cf = apply_constraints_fn(x, cf, hard=False)
        return cf, opt_state

    cf = jnp.array(x, copy=True)
    opt = optax.rmsprop(lr)
    opt_state = opt.init(cf)
    # for _ in tqdm(range(n_steps)):
    #     cf, opt_state = gen_cf_step(x, cf, opt_state)
    cf, opt_state = lax.fori_loop(0, n_steps, gen_cf_step, (cf, opt_state))

    cf = apply_constraints_fn(x, cf, hard=True)
    return cf

In [ ]:
#| export
class VanillaCFConfig(BaseConfig):
    n_steps: int = 100
    lr: float = 0.1
    lambda_: float = 0.1
    validity_fn: str = 'KLDivergence'

In [ ]:
#| export
class VanillaCF(CFModule):

    def __init__(
        self,
        configs: dict | VanillaCFConfig = None,
        name: str = None,
    ):
        if configs is None:
            configs = VanillaCFConfig()
        configs = validate_configs(configs, VanillaCFConfig)
        name = "VanillaCF" if name is None else name
        super().__init__(configs, name=name)

    @auto_reshaping('x')
    def generate_cf(
        self,
        x: Array,  # `x` shape: (k,), where `k` is the number of features
        pred_fn: Callable[[Array], Array],
        y_target: Array = None,
    ) -> jnp.DeviceArray:
        # TODO: Currently assumes binary classification.
        if y_target is None:
            y_target = 1 - pred_fn(x)
        else:
            y_target = jnp.array(y_target, copy=True)

        return _vanilla_cf(
            x=x,  # `x` shape: (k,), where `k` is the number of features
            y_target=y_target,  # `y_target` shape: (1,)
            pred_fn=pred_fn,  # y = pred_fn(x)
            n_steps=self.config.n_steps,
            lr=self.config.lr,  # learning rate for each `cf` optimization step
            lambda_=self.config.lambda_,  #  loss = validity_loss + lambda_params * cost
            validity_fn=keras.losses.get({'class_name': self.config.validity_fn, 'config': {'reduction': None}}),
            cost_fn=keras.losses.get({'class_name': 'MeanSquaredError', 'config': {'reduction': None}}),
            apply_constraints_fn=self.apply_constraints_fn,
        )


In [ ]:
dm = load_data('dummy')
model = load_ml_module('dummy')
xs_train, ys_train = dm['train']
xs_test, ys_test = dm['test']

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [ ]:
vcf = VanillaCF()
vcf.init_apply_fns()
cf = vcf.generate_cf(xs_test[0], model.pred_fn)
assert cf.shape == xs_test[0].shape

partial_gen = partial(vcf.generate_cf, pred_fn=model.pred_fn)
cfs = jax.vmap(partial_gen)(xs_test)

print("Validity: ", keras.metrics.binary_accuracy(
    (1 - model.pred_fn(xs_test)).round(),
    model.pred_fn(cfs)
).mean())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Validity:  0.99600005
